- Importing Libraries

In [13]:
import pandas as pd
import sqlite3 as sql

- Establishing connection

In [14]:
conn = sql.connect("/content/drive/MyDrive/Internship Projects/Week-5/sql-murder-mystery.db")

In [15]:
conn

In [16]:
query_1 = """
SELECT DISTINCT type FROM crime_scene_report;
"""
pd.read_sql_query(query_1, conn)

,type
0,robbery
1,murder
2,theft
3,fraud
4,arson
5,bribery
6,assault
7,smuggling
8,blackmail


In [17]:
query_2 = """
SELECT *
FROM crime_scene_report
WHERE city = "SQL City" AND type = 'murder'
ORDER BY date;
"""

pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_2,conn)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City
1,20180215,murder,REDACTED REDACTED REDACTED,SQL City
2,20180215,murder,Someone killed the guard! He took an arrow to the knee!,SQL City


- Getting details of Witness-1

In [18]:
query_3 = """
SELECT *
FROM person
WHERE address_street_name =  "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1;
"""
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_3,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


- Getting Details of Witness-2

In [19]:
query_4 = """
SELECT *
FROM person
WHERE address_street_name =  "Franklin Ave"
AND name LIKE "%Annabel%";
"""
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_4,conn)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


- Interviews of Two Witnesses

In [20]:
query_5 = """
SELECT *
FROM interview
WHERE person_id IN (14887,16371);
"""
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_5,conn)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


#Finding Killer based on clues got from Interviews

In [21]:
query_6 = """
SELECT p.id,p.name,dl.gender,p.license_id,gfc.membership_id,dl.plate_number,gfc.check_in_date,gfn.membership_start_date,gfn.membership_status  FROM person as p

INNER JOIN drivers_license as dl on dl.id = p.license_id
INNER JOIN get_fit_now_member as gfn on p.id = gfn.person_id
INNER JOIN get_fit_now_check_in as gfc on gfc.membership_id = gfn.id

WHERE plate_number LIKE '%H42W%' AND gender = 'male' AND membership_status = 'gold'
AND gfn.id LIKE '48Z%';
"""
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_6,conn)

,id,name,gender,license_id,membership_id,plate_number,check_in_date,membership_start_date,membership_status
0,67318,Jeremy Bowers,male,423327,48Z55,0H42W2,20180109,20160101,gold


#Found the Killer

In [22]:
cursor = conn.cursor()

query_7 = """
INSERT INTO solution VALUES (1,'Jeremy Bowers');
"""

count = cursor.execute(query_7)
conn.commit()

In [23]:
query_8 = """
SELECT value FROM solution;
"""
pd.read_sql_query(query_8,conn)

,value
0,"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


In [24]:
query_9 = """
SELECT *
FROM interview
WHERE person_id = 67318  ;
"""
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_9,conn)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


# Found the Master Mind of this Mystery - Miranda Priestly

In [25]:
query_10 = """
SELECT p.*,dl.car_make, dl.car_model, dl.hair_color, dl.height, inc.annual_income, fec.event_name
FROM person as p

INNER JOIN income as inc on inc.ssn = p.ssn
INNER JOIN drivers_license as dl on dl.id = p.license_id
INNER JOIN facebook_event_checkin as fec on p.id = fec.person_id

WHERE gender = 'female' AND car_make = 'Tesla' AND car_model = 'Model S'
AND hair_color = 'red';
"""
pd.set_option('display.max_colwidth',None)

pd.read_sql_query(query_10,conn)

,id,name,license_id,address_number,address_street_name,ssn,car_make,car_model,hair_color,height,annual_income,event_name
0,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,Tesla,Model S,red,66,310000,SQL Symphony Concert
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,Tesla,Model S,red,66,310000,SQL Symphony Concert
2,99716,Miranda Priestly,202298,1883,Golden Ave,987756388,Tesla,Model S,red,66,310000,SQL Symphony Concert


In [28]:
cursor = conn.cursor()

query_11 = """
INSERT INTO solution VALUES (1,'Miranda Priestly');
"""

count = cursor.execute(query_11)
conn.commit()

In [29]:
query_12 = """
SELECT value FROM solution;
"""
pd.set_option('display.max_colwidth',None)
pd.read_sql_query(query_12,conn)

,value
0,"Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!"
